<a href="https://colab.research.google.com/github/soham7707/DeepSectoralAI/blob/main/LSTM_Bi_LSTM_GRU_GARCH_Beta_Bullish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install arch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.3/981.3 kB 12.3 MB/s eta 0:00:00


In [ ]:
pip install yfinance pandas numpy matplotlib

In [ ]:
sectors = {
    'IT': ['TCS.NS', 'INFY.NS', 'HCLTECH.NS', 'WIPRO.NS'],
    'Banking': ['HDFCBANK.NS', 'ICICIBANK.NS', 'KOTAKBANK.NS', 'AXISBANK.NS'],
    'Pharma': ['SUNPHARMA.NS', 'CIPLA.NS', 'DRREDDY.NS'],
    'Auto': ['TATAMOTORS.NS', 'M&M.NS', 'MARUTI.NS'],
    'FMCG': ['HINDUNILVR.NS', 'ITC.NS', 'DABUR.NS']
}

In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime

#Bullish Period

start_date = "2020-04-01"
end_date = "2021-10-01"

def fetch_sector_data(sectors):
    all_data = {}
    for sector, stocks in sectors.items():
        print(f"Fetching data for {sector} sector...")
        data = yf.download(stocks, start=start_date, end=end_date)['Close']
        all_data[sector] = data
    return all_data

sector_data = fetch_sector_data(sectors)


Fetching data for IT sector...


/tmp/ipython-input-2431966453.py:14: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, start=start_date, end=end_date)['Close']
[*********************100%***********************]  4 of 4 completed
/tmp/ipython-input-2431966453.py:14: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, start=start_date, end=end_date)['Close']


Fetching data for Banking sector...


[*********************100%***********************]  4 of 4 completed
/tmp/ipython-input-2431966453.py:14: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, start=start_date, end=end_date)['Close']


Fetching data for Pharma sector...


[*********************100%***********************]  3 of 3 completed
/tmp/ipython-input-2431966453.py:14: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, start=start_date, end=end_date)['Close']


Fetching data for Auto sector...


[*********************100%***********************]  3 of 3 completed
/tmp/ipython-input-2431966453.py:14: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stocks, start=start_date, end=end_date)['Close']


Fetching data for FMCG sector...


[*********************100%***********************]  3 of 3 completed


In [ ]:
for sector in sector_data:
    df = sector_data[sector]
    df = df.ffill().bfill()
    sector_data[sector] = df


In [ ]:

import numpy as np

returns_data = {}

for sector, df in sector_data.items():
    returns = np.log(df / df.shift(1)).dropna()
    returns_data[sector] = returns


In [ ]:
for sector, df in returns_data.items():
    df.to_csv(f"{sector}_returns.csv")

In [ ]:
sector_data['Banking']

Ticker,AXISBANK.NS,HDFCBANK.NS,ICICIBANK.NS,KOTAKBANK.NS
Date,,,,
2020-04-01,357.158722,392.912109,300.940399,1176.733887
2020-04-03,324.096802,385.429382,277.244293,1136.103638
2020-04-07,387.233154,424.382050,315.399841,1193.065796
2020-04-08,389.722778,420.972198,308.484467,1182.708862
2020-04-09,418.402985,438.092407,331.455170,1267.604370
...,...,...,...,...
2021-09-24,794.731628,761.757263,701.275635,2020.665161
2021-09-27,793.636292,772.958496,708.114197,2025.895874
2021-09-28,785.918518,768.178345,695.649536,2060.618164


In [ ]:
returns_data['Banking']

Ticker,AXISBANK.NS,HDFCBANK.NS,ICICIBANK.NS,KOTAKBANK.NS
Date,,,,
2020-04-03,-0.097138,-0.019228,-0.082013,-0.035138
2020-04-07,0.177985,0.096276,0.128942,0.048922
2020-04-08,0.006409,-0.008067,-0.022170,-0.008719
2020-04-09,0.071009,0.039863,0.071821,0.069321
2020-04-13,-0.002860,-0.032633,-0.035795,-0.017435
...,...,...,...,...
2021-09-24,-0.017514,0.019896,0.008264,0.002024
2021-09-27,-0.001379,0.014597,0.009704,0.002585
2021-09-28,-0.009772,-0.006203,-0.017759,0.016994


In [ ]:
from arch import arch_model
import pandas as pd
import numpy as np

In [ ]:
def estimate_garch_volatility(returns_series):
    # Fit GARCH(1,1) model to the returns series
    model = arch_model(returns_series, vol='GARCH', p=1, q=1)
    res = model.fit(disp='off')
    # Return the estimated conditional volatility (σ_t)
    return res.conditional_volatility

In [ ]:
garch_volatility_data = {}

for sector, returns_df in returns_data.items():
    print(f"Estimating GARCH volatility for {sector} sector...")
    sector_vols = pd.DataFrame(index=returns_df.index)

    for stock in returns_df.columns:
        try:
            vol = estimate_garch_volatility(returns_df[stock])
            sector_vols[stock] = vol
        except Exception as e:
            print(f"Error processing {stock}: {e}")

    garch_volatility_data[sector] = sector_vols

Estimating GARCH volatility for IT sector...


/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:694: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0003924. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  self._check_scale(resids)
/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:694: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0002982. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  self._check_scale(resids)
/usr/local/lib/python3.12/dist-packages/arch/univariate/base

Estimating GARCH volatility for Banking sector...
Estimating GARCH volatility for Pharma sector...


/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:694: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0008205. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  self._check_scale(resids)
/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:694: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0003756. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  self._check_scale(resids)
/usr/local/lib/python3.12/dist-packages/arch/univariate/base

Estimating GARCH volatility for Auto sector...
Estimating GARCH volatility for FMCG sector...


/usr/local/lib/python3.12/dist-packages/arch/univariate/base.py:694: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0002866. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  self._check_scale(resids)


In [ ]:
pip install numpy pandas scikit-learn tensorflow

In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def prepare_lstm_data(series, lookback=20):
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(series.values.reshape(-1, 1))

    X, y = [], []
    for i in range(lookback, len(scaled_data)):
        X.append(scaled_data[i-lookback:i])
        y.append(scaled_data[i])

    X = np.array(X)
    y = np.array(y)
    return X, y, scaler

In [ ]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=False, input_shape=input_shape))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

In [ ]:
def predict_lstm_volatility(returns_series, lookback=20):
    X, y, scaler = prepare_lstm_data(returns_series, lookback)
    model = build_lstm_model((X.shape[1], X.shape[2]))
    model.fit(X, y, epochs=10, batch_size=16, verbose=0)

    y_pred_scaled = model.predict(X)
    y_pred = scaler.inverse_transform(y_pred_scaled).flatten()

    # Estimate volatility as rolling std of predicted returns
    predicted_volatility = pd.Series(y_pred).rolling(window=lookback).std()
    predicted_volatility.index = returns_series.index[lookback:]
    return predicted_volatility

In [ ]:
lstm_volatility_data = {}

for sector, returns_df in returns_data.items():
    print(f"Predicting LSTM volatility for {sector} sector...")
    sector_vols = pd.DataFrame()

    for stock in returns_df.columns:
        try:
            vol = predict_lstm_volatility(returns_df[stock])
            sector_vols[stock] = vol
        except Exception as e:
            print(f"Error with {stock}: {e}")

    lstm_volatility_data[sector] = sector_vols

Predicting LSTM volatility for IT sector...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step


...

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step  


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step  


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step  


In [ ]:
from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.layers import LSTM, Dense, Bidirectional

def build_bilstm_model(input_shape):
    model = Sequential()
    model.add(Bidirectional(LSTM(units=50, return_sequences=False), input_shape=input_shape))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

In [ ]:
def predict_bilstm_volatility(returns_series, lookback=20):
    X, y, scaler = prepare_lstm_data(returns_series, lookback)
    model = build_bilstm_model((X.shape[1], X.shape[2]))
    model.fit(X, y, epochs=10, batch_size=16, verbose=0)

    y_pred_scaled = model.predict(X)
    y_pred = scaler.inverse_transform(y_pred_scaled).flatten()

    predicted_volatility = pd.Series(y_pred).rolling(window=lookback).std()
    predicted_volatility.index = returns_series.index[lookback:]
    return predicted_volatility

In [ ]:
bilstm_volatility_data = {}

for sector, returns_df in returns_data.items():
    print(f"Predicting BiLSTM volatility for {sector} sector...")
    sector_vols = pd.DataFrame()

    for stock in returns_df.columns:
        try:
            vol = predict_bilstm_volatility(returns_df[stock])
            sector_vols[stock] = vol
        except Exception as e:
            print(f"Error with {stock}: {e}")

    bilstm_volatility_data[sector] = sector_vols

Predicting BiLSTM volatility for IT sector...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step  


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step  


...

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step  


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step  


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step  


In [ ]:
from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.layers import LSTM, Dense, Bidirectional
from tensorflow.keras.layers import GRU, Dense

def build_gru_model(input_shape):
    model = Sequential()
    model.add(GRU(units=50, return_sequences=False, input_shape=input_shape))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model


In [ ]:
def predict_gru_volatility(returns_series, lookback=20):
    X, y, scaler = prepare_lstm_data(returns_series, lookback)  # same prepare function works
    model = build_gru_model((X.shape[1], X.shape[2]))
    model.fit(X, y, epochs=10, batch_size=16, verbose=0)

    y_pred_scaled = model.predict(X)
    y_pred = scaler.inverse_transform(y_pred_scaled).flatten()

    predicted_volatility = pd.Series(y_pred).rolling(window=lookback).std()
    predicted_volatility.index = returns_series.index[lookback:]
    return predicted_volatility

In [ ]:
#GRU
gru_volatility_data = {}

for sector, returns_df in returns_data.items():
    print(f"Predicting GRU volatility for {sector} sector...")
    sector_vols = pd.DataFrame()

    for stock in returns_df.columns:
        try:
            vol = predict_gru_volatility(returns_df[stock])
            sector_vols[stock] = vol
        except Exception as e:
            print(f"Error with {stock}: {e}")

    gru_volatility_data[sector] = sector_vols

Predicting GRU volatility for IT sector...


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step  


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step  


...

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step  


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step  


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step  


In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd

# Download NIFTY-50 index (symbol: ^NSEI)
nifty = yf.download("^NSEI", start="2015-01-01")['Close']
nifty_returns = np.log(nifty / nifty.shift(1)).dropna()
nifty_returns_df = pd.DataFrame(nifty_returns)
nifty_returns_df.columns = ['Market']

/tmp/ipython-input-2614447981.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  nifty = yf.download("^NSEI", start="2015-01-01")['Close']
[*********************100%***********************]  1 of 1 completed


In [ ]:
import statsmodels.api as sm

def compute_beta(stock_returns, market_returns_df):
    aligned = pd.concat([stock_returns, market_returns_df], axis=1).dropna()
    aligned.columns = ['Stock', 'Market']

    X = sm.add_constant(aligned['Market'])
    y = aligned['Stock']
    model = sm.OLS(y, X).fit()
    return model.params['Market']  # This is the beta

beta_values = {}

for sector, returns_df in returns_data.items():
    print(f"Computing Beta for {sector} sector...")
    sector_betas = {}

    for stock in returns_df.columns:
        try:
            beta = compute_beta(returns_df[stock], nifty_returns_df)
            sector_betas[stock] = beta
        except Exception as e:
            print(f"Error with {stock}: {e}")

    beta_values[sector] = sector_betas



beta_df = pd.DataFrame(beta_values).T  # Transpose to have stocks as columns
beta_df = beta_df.fillna(0)
print(beta_df)


Computing Beta for IT sector...
Computing Beta for Banking sector...
Computing Beta for Pharma sector...
Computing Beta for Auto sector...
Computing Beta for FMCG sector...
         HCLTECH.NS   INFY.NS    TCS.NS  WIPRO.NS  AXISBANK.NS  HDFCBANK.NS  \
IT         0.774226  0.745206  0.695667  0.697243     0.000000     0.000000   
Banking    0.000000  0.000000  0.000000  0.000000     1.731113     1.252224   
Pharma     0.000000  0.000000  0.000000  0.000000     0.000000     0.000000   
Auto       0.000000  0.000000  0.000000  0.000000     0.000000     0.000000   
FMCG       0.000000  0.000000  0.000000  0.000000     0.000000     0.000000   

         ICICIBANK.NS  KOTAKBANK.NS  CIPLA.NS  DRREDDY.NS  SUNPHARMA.NS  \
IT           0.000000      0.000000  0.000000    0.000000      0.000000   
Banking      1.746477      1.136998  0.000000    0.000000      0.000000   
Pharma       0.000000      0.000000  0.357097    0.369719      0.562616   
Auto         0.000000      0.000000  0.000000    0.0

In [ ]:
sharpe_ratios = {}

for sector, df in returns_data.items():
    sector_sharpes = {}
    for stock in df.columns:
        r = df[stock].dropna()
        mean_return = r.mean()
        std_return = r.std()
        sharpe = mean_return / std_return if std_return != 0 else 0
        sector_sharpes[stock] = sharpe
    sharpe_ratios[sector] = sector_sharpes



features = []

for sector in returns_data:
    for stock in returns_data[sector].columns:
        try:
            beta = beta_values[sector][stock]
            garch_mean = garch_volatility_data[sector][stock].mean()
            lstm_mean = lstm_volatility_data[sector][stock].mean()
            bilstm_mean = bilstm_volatility_data[sector][stock].mean()
            gru_mean = gru_volatility_data[sector][stock].mean()
            sharpe = sharpe_ratios[sector][stock]

            features.append([sector, stock, beta, garch_mean, lstm_mean, bilstm_mean, gru_mean, sharpe])
        except Exception as e:
            print(f"Skipping {stock} due to missing data: {e}")

features_df = pd.DataFrame(
    features,
    columns=['Sector', 'Stock', 'Beta', 'GARCH_Vol', 'LSTM_Vol', 'BiLSTM_Vol', 'GRU_Vol', 'Sharpe']
)


In [ ]:
features_df = pd.DataFrame(features, columns=['Sector', 'Stock', 'Beta', 'GARCH_Vol','LSTM_Vol', 'BiLSTM_Vol', 'GRU_Vol', 'Sharpe'])

In [ ]:
from sklearn.linear_model import LinearRegression

X = features_df[['Beta', 'GARCH_Vol', 'GRU_Vol','LSTM_Vol']]
y = features_df['Sharpe']

model = LinearRegression().fit(X, y)
weights = model.coef_
intercept = model.intercept_

print("Learned Weights:", weights)
print("Intercept:", intercept)

Learned Weights: [ -0.04300349   5.26484707  14.30765957 -29.30237698]
Intercept: 0.05048755190625591


In [ ]:
X = features_df[['Beta', 'GARCH_Vol', 'GRU_Vol','LSTM_Vol']]
y = features_df['Sharpe']

# Drop rows with NaNs
df_clean = pd.concat([X, y], axis=1).dropna()

X = df_clean[['Beta', 'GARCH_Vol', 'GRU_Vol','LSTM_Vol']]
y = df_clean['Sharpe']

model = LinearRegression().fit(X, y)
print("Learned Weights:", model.coef_)
print("Intercept:", model.intercept_)

Learned Weights: [ -0.04300349   5.26484707  14.30765957 -29.30237698]
Intercept: 0.05048755190625591


In [ ]:
features_df['Risk_Score'] = X @ weights

In [ ]:
from sklearn.linear_model import LinearRegression

X = features_df[['Beta', 'GARCH_Vol', 'GRU_Vol','BiLSTM_Vol']]
y = features_df['Sharpe']

model = LinearRegression().fit(X, y)
weights = model.coef_
intercept = model.intercept_

print("Learned Weights:", weights)
print("Intercept:", intercept)


Learned Weights: [-2.63997509e-02  1.48021370e+00 -9.02374668e+00  4.87525994e+01]
Intercept: 0.04615044037119165


In [ ]:
features_df['Risk_Score_GRU_Bi_Beta_GARCH'] = X @ weights

In [ ]:
for sector in features_df['Sector'].unique():
    print(f"\nSafest stocks in {sector} (beta + GARCH + GRU+ bilstm):")
    print(features_df[features_df['Sector'] == sector]
          .sort_values('Risk_Score_GRU_Bi_Beta_GARCH')
          .head(3)[['Stock', 'Risk_Score_GRU_Bi_Beta_GARCH']])



Safest stocks in IT (beta + GARCH + GRU+ bilstm):
        Stock  Risk_Score_GRU_Bi_Beta_GARCH
2      TCS.NS                      0.051996
0  HCLTECH.NS                      0.056816
1     INFY.NS                      0.062357

Safest stocks in Banking (beta + GARCH + GRU+ bilstm):
          Stock  Risk_Score_GRU_Bi_Beta_GARCH
6  ICICIBANK.NS                      0.034328
7  KOTAKBANK.NS                      0.039299
5   HDFCBANK.NS                      0.051715

Safest stocks in Pharma (beta + GARCH + GRU+ bilstm):
           Stock  Risk_Score_GRU_Bi_Beta_GARCH
10  SUNPHARMA.NS                      0.056338
8       CIPLA.NS                      0.063794
9     DRREDDY.NS                      0.080578

Safest stocks in Auto (beta + GARCH + GRU+ bilstm):
            Stock  Risk_Score_GRU_Bi_Beta_GARCH
11         M&M.NS                      0.045897
12      MARUTI.NS                      0.054197
13  TATAMOTORS.NS                      0.084754

Safest stocks in FMCG (beta + GARCH + GRU+ b

In [ ]:
from sklearn.linear_model import LinearRegression

X = features_df[['Beta', 'GARCH_Vol', 'LSTM_Vol','BiLSTM_Vol']]
y = features_df['Sharpe']

model = LinearRegression().fit(X, y)
weights = model.coef_
intercept = model.intercept_

print("Learned Weights:", weights)
print("Intercept:", intercept)

Learned Weights: [-4.69309706e-02  4.27145040e+00 -6.16119091e+01  9.01148636e+01]
Intercept: 0.03395303379657216


In [ ]:
features_df['Risk_Score_lstm_bi_garch_Beta'] = X @ weights

In [ ]:
for sector in features_df['Sector'].unique():
    print(f"\nSafest stocks in {sector} (LSTM+ BiLSTM + GARCH + Beta ):")
    print(features_df[features_df['Sector'] == sector]
          .sort_values('Risk_Score_lstm_bi_garch_Beta')
          .head(3)[['Stock', 'Risk_Score_lstm_bi_garch_Beta']])



Safest stocks in IT (LSTM+ BiLSTM + GARCH + Beta ):
        Stock  Risk_Score_lstm_bi_garch_Beta
0  HCLTECH.NS                       0.067009
2      TCS.NS                       0.077280
1     INFY.NS                       0.084830

Safest stocks in Banking (LSTM+ BiLSTM + GARCH + Beta ):
          Stock  Risk_Score_lstm_bi_garch_Beta
6  ICICIBANK.NS                       0.042442
7  KOTAKBANK.NS                       0.045305
5   HDFCBANK.NS                       0.046395

Safest stocks in Pharma (LSTM+ BiLSTM + GARCH + Beta ):
           Stock  Risk_Score_lstm_bi_garch_Beta
8       CIPLA.NS                       0.069431
10  SUNPHARMA.NS                       0.069966
9     DRREDDY.NS                       0.073856

Safest stocks in Auto (LSTM+ BiLSTM + GARCH + Beta ):
            Stock  Risk_Score_lstm_bi_garch_Beta
12      MARUTI.NS                       0.048123
11         M&M.NS                       0.052632
13  TATAMOTORS.NS                       0.101152

Safest stocks in FMC

In [ ]:
from sklearn.linear_model import LinearRegression

X = features_df[['Beta', 'GARCH_Vol', 'GRU_Vol','LSTM_Vol']]
y = features_df['Sharpe']

model = LinearRegression().fit(X, y)
weights = model.coef_
intercept = model.intercept_

print("Learned Weights:", weights)
print("Intercept:", intercept)

Learned Weights: [ -0.04300349   5.26484707  14.30765957 -29.30237698]
Intercept: 0.05048755190625591


In [ ]:
from sklearn.linear_model import LinearRegression

X = features_df[['Beta', 'GARCH_Vol', 'LSTM_Vol','BiLSTM_Vol', 'GRU_Vol']]
y = features_df['Sharpe']

model = LinearRegression().fit(X, y)
weights = model.coef_
intercept = model.intercept_

print("Learned Weights:", weights)
print("Intercept:", intercept)

Learned Weights: [-4.95857507e-02  4.48421008e+00 -6.60049839e+01  8.69415263e+01
  4.74730223e+00]
Intercept: 0.03297809497758762


In [ ]:
features_df['Risk_Score_LSTM_Bi-LSTM_GRU_GARCH_Beta'] = X @ weights

In [ ]:
for sector in features_df['Sector'].unique():
    print(f"\nSafest stocks in {sector} (GARCH + Beta + GRU + LSTM + Bi-LSTM):")
    print(features_df[features_df['Sector'] == sector]
          .sort_values('Risk_Score_LSTM_Bi-LSTM_GRU_GARCH_Beta')
          .head(3)[['Stock', 'Risk_Score_LSTM_Bi-LSTM_GRU_GARCH_Beta']])


Safest stocks in IT (GARCH + Beta + GRU + LSTM + Bi-LSTM):
        Stock  Risk_Score_LSTM_Bi-LSTM_GRU_GARCH_Beta
0  HCLTECH.NS                                0.065918
2      TCS.NS                                0.078130
1     INFY.NS                                0.085703

Safest stocks in Banking (GARCH + Beta + GRU + LSTM + Bi-LSTM):
          Stock  Risk_Score_LSTM_Bi-LSTM_GRU_GARCH_Beta
5   HDFCBANK.NS                                0.045035
6  ICICIBANK.NS                                0.047157
7  KOTAKBANK.NS                                0.047915

Safest stocks in Pharma (GARCH + Beta + GRU + LSTM + Bi-LSTM):
           Stock  Risk_Score_LSTM_Bi-LSTM_GRU_GARCH_Beta
8       CIPLA.NS                                0.069352
10  SUNPHARMA.NS                                0.072007
9     DRREDDY.NS                                0.072789

Safest stocks in Auto (GARCH + Beta + GRU + LSTM + Bi-LSTM):
            Stock  Risk_Score_LSTM_Bi-LSTM_GRU_GARCH_Beta
12      MARUTI.NS      

In [ ]:
features_df[['Stock', 'Risk_Score', 'Risk_Score_LSTM_Bi-LSTM_GRU_GARCH_Beta']].head()

,Stock,Risk_Score,Risk_Score_LSTM_Bi-LSTM_GRU_GARCH_Beta
0,HCLTECH.NS,0.044800,0.065918
1,INFY.NS,0.044368,0.085703
2,TCS.NS,0.046368,0.078130
3,WIPRO.NS,0.069660,0.099196
4,AXISBANK.NS,0.050715,0.094388
